In [0]:
DEBUG = False
CONFIG_FILE = '/datascope/subaru/data/targeting/dSph/draco/netflow/SSP/draco_6_1_007/ga-netflow_20250315155504.config'
OUTPUT_PATH = '/datascope/subaru/data/targeting/dSph/draco/netflow/SSP/draco_6_1_007'

# Plot the assigned targets

Load the data and plot the assigned targets for each pointing and visit.

In [0]:
import os, sys
from glob import glob
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [0]:
plt.rc('font', size=6)

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
if DEBUG and 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', int(os.environ['PFS_TARGETING_DEBUGPORT'])))
    debug = True

# Imports

In [0]:
import pfs.utils
from pfs.datamodel import TargetType

from pfs.ga.targeting.config.netflow import NetflowConfig
from pfs.ga.targeting.scripts.netflow.netflowscript import NetflowScript
from pfs.ga.targeting.io import DataFrameSerializer, ObservationSerializer
from pfs.ga.targeting.targets.dsph import *
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import WcsProjection, Pointing
from pfs.ga.targeting.netflow import Netflow
from pfs.ga.targeting.io import DataFrameSerializer

# Load the netflow config file

In [0]:
# Load the configuration
config = NetflowConfig.default()
config.load(CONFIG_FILE, ignore_collisions=True, format='.json')

print(config.targets.keys())

# Load the target lists

In [0]:
target_lists = {}

for key in config.targets:
    fn = os.path.join(OUTPUT_PATH, f'{config.field.key}_targets_{key}.feather')
    target_lists[key] = NetflowScript.load_target_list(key, config.targets[key], fn)
    print(key, config.targets[key].prefix, config.targets[key].path)

In [0]:
# Print the priority classes for each target list
for k, target_list in target_lists.items():
    print(k)
    if 'priority' in target_list.data:
        print(target_list.data['priority'].unique())

In [0]:
# List available photometry for each target list
for k, target_list in target_lists.items():
    print(k)
    for p in target_list.photometry:
        print(' ', p)
        for m in target_list.photometry[p].magnitudes:
            print('    ', m)

In [0]:
for k, target_list in target_lists.items():
    print(k)
    print(target_list.data.columns)

# Load the assignments

In [0]:
# Load the assignments

fn = os.path.join(OUTPUT_PATH, f'{config.field.key}_assignments.feather')
assignments = DataFrameSerializer().read(fn)

print(assignments.shape)
for c in assignments.columns:
    print(c, assignments[c].dtype)

In [0]:
# Print the unique pointing and visit indices
assignments['pointing_idx'].unique(), assignments['visit_idx'].unique()

In [0]:
# Load the assignment summary

fn = os.path.join(OUTPUT_PATH, f'{config.field.key}_summary.feather')
summary = DataFrameSerializer().read(fn)

print(summary.shape)
for c in assignments.columns:
    print(c, assignments[c].dtype)

# Plot assigned targets

In [0]:
pfi = SubaruPFI(instrument_options=config.instrument_options)

In [0]:
def get_photometry_all():
    # Get unique photometric systems from all target lists
    photometry = {}
    for k, target_list in target_lists.items():
        for p, phot in target_list.photometry.items():
            if p not in photometry and len(phot.magnitudes) > 1:
                    photometry[p] = phot

    return photometry

def create_fov(pointing):
    wcs = WcsProjection(pointing, proj='TAN')
    wfc = SubaruWFC(pointing)
    fov = FOV(projection=wcs)

    return wcs, fov

def create_cmd(photometry):
    # Create a color-magnitude diagram using the first two filters of the photometric systems
    cmd = {}
    for p, phot in photometry.items():
        if len(phot.magnitudes) > 1:
            mm = list(phot.magnitudes.keys())
            m1, m2 = phot.magnitudes[mm[0]], phot.magnitudes[mm[1]]
            cmd[p] = CMD([ColorAxis(Color([m1, m2])), MagnitudeAxis(m2)])
    return cmd

def plot_assignments(f, gs, cmd, target_lists,
                     pointing, pointing_idx=None, visit_idx=None,
                     plot_background=False, plot_all=True, plot_assigned=False, plot_unassigned=False,
                     color_by_priority=False, color_by_target_list=False,
                     priority=None,
                     title=None):
    
    cmap = plt.get_cmap('tab10')

    # Create the field-of-view plot
    wcs, fov = create_fov(pointing)

    # Create subplots
    axs = [ f.add_subplot(gs[0], projection=wcs.wcs) ]
    for i, c in enumerate(cmd):
        axs.append(f.add_subplot(gs[i + 1]))

    # Filter for the current visit
    mask = ((pointing_idx is None) | (assignments['pointing_idx'] == pointing_idx)) \
         & ((visit_idx is None) | (assignments['visit_idx'] == visit_idx))

    unique_targets = assignments['target_idx'][mask].unique()
    # print(unique_targets.shape)

    # Plot the full list of science targets in the background in grey
    for j, (k, target_list) in enumerate(target_lists.items()):
        if config.targets[k].prefix == 'sci':
            for i in range(4):
                if i == 0 and plot_background:
                    mask = None
                    color = 'lightgrey'
                    alpha = 0.3
                    scalex, scaley = False, False
                elif i == 0:
                    continue

                if i == 1 and plot_all:
                    mask = None
                    alpha = 1.0
                    scalex, scaley = True, True
                elif i == 1:
                    continue

                if i == 2 and plot_assigned:
                    mask = target_list.data['__target_idx'].isin(unique_targets)
                    alpha = 1.0
                    scalex, scaley = True, True
                elif i == 2:
                    continue

                if i == 3 and plot_unassigned:
                    mask = ~target_list.data['__target_idx'].isin(unique_targets)
                    alpha = 1.0
                    scalex, scaley = True, True
                elif i == 3:
                    continue

                if i > 0 and priority is not None:
                    priority_mask = target_list.data['priority'].isin(priority)
                    if mask is None:
                        mask = priority_mask
                    else:
                        mask &= priority_mask

                if i > 0:
                    if color_by_priority:
                        color = cmap(target_list.data['priority'].values[mask])
                    elif color_by_target_list:
                        color = cmap(j)
                    else:
                        color = 'red'

                fov.plot_catalog(axs[0], target_list, mask=mask, observed=True, c=color, alpha=alpha, size=0.5,
                                 scalex=scalex, scaley=scaley)

                for i, (_, c) in enumerate(cmd.items()):
                    if target_list.has_diagram_values(c, observed=True):
                        c.plot_catalog(axs[i + 1], target_list, mask=mask, observed=True, c=color, alpha=alpha, size=0.5)

        # if target_list.has_diagram_values(ccd, observed=True):
        #     ccd.plot_catalog(axs[1], target_list, observed=True, color='grey', alpha=1.0)

    # ax.set_title(target_list.name)
    axs[0].set_aspect('equal', adjustable='datalim')
    axs[0].set_xlim(1.5, -2)
    axs[0].set_ylim(-2, 1.0)
    axs[0].legend(loc='upper right', fontsize=6)

    for i, (c, _) in enumerate(cmd.items()):
        axs[i + 1].set_title(c)

In [0]:
# Collect all photometric systems and create the corresponding CMDs
photometry = get_photometry_all()
cmd = create_cmd(photometry)

f = plt.figure(figsize=(3 + 2 * len(cmd), 2.5), dpi=240)
f.suptitle('All targets')
gs = f.add_gridspec(1, len(cmd) + 1, width_ratios=[3,] + len(cmd) * [2,], wspace=0.4, hspace=0.2)
plot_assignments(f, gs, cmd, target_lists,
                 config.pointings[0].get_pointing(),
                 plot_background=False, plot_all=True, plot_assigned=False, plot_unassigned=False)

f = plt.figure(figsize=(3 + 2 * len(cmd), 2.5), dpi=240)
f.suptitle('Assigned targets')
gs = f.add_gridspec(1, len(cmd) + 1, width_ratios=[3,] + len(cmd) * [2,], wspace=0.4, hspace=0.2)

plot_assignments(f, gs, cmd, target_lists,
                 config.pointings[0].get_pointing(),
                 plot_background=True, plot_all=False, plot_assigned=True, plot_unassigned=False)

f = plt.figure(figsize=(3 + 2 * len(cmd), 2.5), dpi=240)
f.suptitle('Unassigned targets')
gs = f.add_gridspec(1, len(cmd) + 1, width_ratios=[3,] + len(cmd) * [2,], wspace=0.4, hspace=0.2)

plot_assignments(f, gs, cmd, target_lists,
                 config.pointings[0].get_pointing(),
                 plot_background=True, plot_all=False, plot_assigned=False, plot_unassigned=True)

# Plot science targets per priority

In [0]:
# Collect all photometric systems and create the corresponding CMDs
photometry = get_photometry_all()
cmd = create_cmd(photometry)

f = plt.figure(figsize=(3 + 2 * len(cmd), 2.5), dpi=240)
f.suptitle('Assigned targets')
gs = f.add_gridspec(1, len(cmd) + 1, width_ratios=[3,] + len(cmd) * [2,], wspace=0.4, hspace=0.2)

plot_assignments(f, gs, cmd, target_lists,
                 config.pointings[0].get_pointing(),
                 plot_background=True, plot_all=False, plot_assigned=True, plot_unassigned=False,
                 color_by_priority=True)


In [0]:
mask = (assignments['prefix'] == 'sci') & ~assignments['priority'].isna()
for pp in np.sort(assignments['priority'][mask].unique()):

    f = plt.figure(figsize=(3 + 2 * len(cmd), 2.5), dpi=240)
    f.suptitle(f'Assigned targets with priority {pp}')
    gs = f.add_gridspec(1, len(cmd) + 1, width_ratios=[3,] + len(cmd) * [2,], wspace=0.4, hspace=0.2)

    plot_assignments(f, gs, cmd, target_lists,
                    config.pointings[0].get_pointing(),
                    plot_background=True, plot_all=False, plot_assigned=True, plot_unassigned=False,
                    priority=[pp], color_by_priority=False)


In [0]:
# Collect all photometric systems and create the corresponding CMDs
photometry = get_photometry_all()
cmd = create_cmd(photometry)

f = plt.figure(figsize=(3 + 2 * len(cmd), 2.5), dpi=240)
f.suptitle('Unassigned targets')
gs = f.add_gridspec(1, len(cmd) + 1, width_ratios=[3,] + len(cmd) * [2,], wspace=0.4, hspace=0.2)

plot_assignments(f, gs, cmd, target_lists,
                 config.pointings[0].get_pointing(),
                 plot_background=True, plot_all=False, plot_assigned=False, plot_unassigned=True,
                 color_by_priority=True)


In [0]:
mask = (assignments['prefix'] == 'sci') & ~assignments['priority'].isna()
for pp in np.sort(assignments['priority'][mask].unique()):

    f = plt.figure(figsize=(3 + 2 * len(cmd), 2.5), dpi=240)
    f.suptitle(f'Unassigned targets with priority {pp}')
    gs = f.add_gridspec(1, len(cmd) + 1, width_ratios=[3,] + len(cmd) * [2,], wspace=0.4, hspace=0.2)

    plot_assignments(f, gs, cmd, target_lists,
                    config.pointings[0].get_pointing(),
                    plot_background=True, plot_all=False, plot_assigned=False, plot_unassigned=True,
                    priority=[pp], color_by_priority=False)


# Calculate statistics

In [0]:
# Total number of unique science targets observed
assignments[assignments['prefix'] == 'sci']['targetid'].nunique()

In [0]:
summary['num_visits'][summary['num_visits'] > 0]

In [0]:
# Science targets that are partially observed
summary[(summary['prefix'] == 'sci') & (summary['done_visits'] < summary['req_visits']) & (summary['num_visits'] > 0)]

In [0]:
# Any target that is partially observed
summary[(summary['done_visits'] < summary['req_visits']) & (summary['num_visits'] > 0)]

In [0]:
# Science targets that are observed longer than the required number of visits
summary[(summary['prefix'] == 'sci') & (summary['num_visits'] > summary['req_visits'])]

In [0]:
# Number of unique targets per priority class
summary[summary['num_visits'] > 0].groupby('class')['targetid'].nunique()

In [0]:
# Number of missed science targets per priority class
summary[(summary['prefix'] == 'sci') & (summary['num_visits'] == 0)].groupby('class')['targetid'].nunique()

In [0]:
# Number of partially observed science targets per priority class
summary[(summary['prefix'] == 'sci') & 
        (summary['done_visits'] < summary['req_visits']) & 
        (summary['done_visits'] > 0)].groupby('class')['targetid'].nunique()

In [0]:
# Number of unassigned fibers in each visit
assignments[assignments['target_type'] == TargetType.UNASSIGNED].groupby('visit_idx')['fiberid'].nunique()

In [0]:
# Number of unique calibration targets for each visit
assignments[assignments['prefix'] == 'cal'].groupby('visit_idx')['targetid'].nunique()

In [0]:
# Number of unique sky positions for each visit
assignments[assignments['prefix'] == 'sky'].groupby('visit_idx')['targetid'].nunique()

In [0]:
# Number of observed unique science targets for each visit
assignments[assignments['prefix'] == 'sci'].groupby('visit_idx')['targetid'].nunique()

In [0]:
# of unique science targets assigned
for pidx in np.sort(assignments['pointing_idx'].unique()):
    for vidx in np.sort(assignments[assignments['pointing_idx'] == pidx]['visit_idx'].unique()):
        mask = (assignments['visit_idx'] == vidx) & \
               (assignments['pointing_idx'] == pidx) & \
               (assignments['prefix'] == 'sci')
        print(f'Pointing #{pidx}, Visit #{vidx}:', assignments[mask]['targetid'].nunique())

print('Grand Total:', assignments[assignments['prefix'] == 'sci']['targetid'].nunique())

In [0]:
# total # of sky fibers assigned
for pidx in np.sort(assignments['pointing_idx'].unique()):
    for vidx in np.sort(assignments[assignments['pointing_idx'] == pidx]['visit_idx'].unique()):
        mask = (assignments['visit_idx'] == vidx) & \
               (assignments['pointing_idx'] == pidx) & \
               (assignments['prefix'] == 'sky')
        print(f'Pointing #{pidx}, Visit #{vidx}:', assignments[mask]['targetid'].nunique())

print('Grand Total:', assignments[assignments['prefix'] == 'sky']['targetid'].nunique())

In [0]:
# total # of calibration fibers assigned
for pidx in np.sort(assignments['pointing_idx'].unique()):
    for vidx in np.sort(assignments[assignments['pointing_idx'] == pidx]['visit_idx'].unique()):
        mask = (assignments['visit_idx'] == vidx) & \
               (assignments['pointing_idx'] == pidx) & \
               (assignments['prefix'] == 'cal')
        print(f'Pointing #{pidx}, Visit #{vidx}:', assignments[mask]['targetid'].nunique())

print('Grand Total:', assignments[assignments['prefix'] == 'cal']['targetid'].nunique())

# Plot the distribution of assigned targets

In [0]:
# Plot priority distribution for each target list
for k in config.targets:
    if config.targets[k].prefix in ['sci']:
        f, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=240)

        hist = np.bincount(target_lists[k].data['priority'])
        bins = np.arange(hist.size)
        ax.bar(bins, hist)

        # print(k, 'all', hist)

        mask = target_lists[k].data['__target_idx'].isin(assignments[assignments['prefix'] == config.targets[k].prefix]['target_idx'])
        hist2 = np.bincount(target_lists[k].data['priority'][mask])
        bins2 = np.arange(hist2.size)
        ax.bar(bins2, hist2, color='r')

        # print(k, 'assigned', hist2)

        # Add the number of targets to the top of each bar
        for i, (v, v2) in enumerate(zip(hist, hist2)):
            ax.text(i, v + 1, f'{v}\n{v2}', ha='center', va='bottom')

        ax.set_title(f'Priority class distribution for target list `{k}`')
        ax.set_xlabel(f'Priority class')
        ax.set_ylabel(f'Target count')

In [0]:
# Plot priority distribution for each target list
for k in config.targets:
    if config.targets[k].prefix in ['sci']:
        f, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=240)

        hist_all = np.bincount(target_lists[k].data['priority'])

        mask = target_lists[k].data['__target_idx'].isin(assignments[assignments['prefix'] == config.targets[k].prefix]['target_idx'])
        hist = np.zeros_like(hist_all)
        h = np.bincount(target_lists[k].data['priority'][mask])
        hist[:h.size] = h
        
        ax.bar(np.arange(hist.size), np.full(hist.size, 1))
        ax.bar(np.arange(hist.size), hist / hist_all, color='r')

        # Add the fraction of targets to the top of each bar
        for i, v in enumerate(hist / hist_all):
            ax.text(i, 1.01, f'{v:0.2f}', ha='center', va='bottom')

        ax.set_title(f'Priority class distribution for target list `{k}`')
        ax.set_xlabel(f'Priority class')
        ax.set_ylabel(f'Assigned fraction')

        ax.set_ylim(0, 1.1)

In [0]:
# Plot the distribution of required visits for each target list
exp_time = 1800 # seconds

for k in config.targets:
    if config.targets[k].prefix in ['sci']:
        f, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=240)

        hist = np.bincount((np.ceil(target_lists[k].data['exp_time'] / exp_time)).astype(int))
        bins = np.arange(hist.size)
        ax.bar(bins, hist)

        # print(k, 'all', hist)

        mask = target_lists[k].data['__target_idx'].isin(assignments[assignments['prefix'] == config.targets[k].prefix]['target_idx'])
        hist2 = np.bincount((np.ceil(target_lists[k].data['exp_time'][mask] / exp_time)).astype(int))
        bins2 = np.arange(hist2.size)
        ax.bar(bins2, hist2, color='r')

        # print(k, 'assigned', hist2)

        # Add the number of targets to the top of each bar
        for i, (v, v2) in enumerate(zip(hist, hist2)):
            ax.text(i, v + 1, f'{v}\n{v2}', ha='center', va='bottom')

        ax.set_title(f'Distribution of required visit (t_exp = {exp_time} s)')
        ax.set_xlabel('Number of required visits')
        ax.set_ylabel('Target count')


# Distribution of black dot distance

In [0]:
# total # of sky fibers assigned
for pidx in np.sort(assignments['pointing_idx'].unique()):
    for vidx in np.sort(assignments[assignments['pointing_idx'] == pidx]['visit_idx'].unique()):
        f, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=240)

        mask = ((assignments['target_type'] != TargetType.UNASSIGNED) &
                (assignments['target_type'] != TargetType.ENGINEERING))
        
        bins = np.linspace(0, 10, 50)
        hist, _ = np.histogram(assignments[mask & (assignments['visit_idx'] == vidx)]['black_dot_dist'], bins=bins)

        ax.step(0.5 * (bins[1:] + bins[:-1]), hist, where='mid')

        ax.set_title(f'Distribution of black dot dist for pointing #{pidx}, visit #{vidx}')
        ax.set_xlabel('Black dot distance [mm]')
        ax.set_ylabel('Fiber count')

# Distribution of cobra center distance

In [0]:
# total # of sky fibers assigned
for pidx in np.sort(assignments['pointing_idx'].unique()):
    for vidx in np.sort(assignments[assignments['pointing_idx'] == pidx]['visit_idx'].unique()):
        f, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=240)

        mask = ((assignments['target_type'] != TargetType.UNASSIGNED) &
                (assignments['target_type'] != TargetType.ENGINEERING))
        
        bins = np.linspace(0, 10, 50)
        hist, _ = np.histogram(assignments[mask & (assignments['visit_idx'] == vidx)]['center_dist'], bins=bins)

        ax.step(0.5 * (bins[1:] + bins[:-1]), hist, where='mid')

        ax.set_title(f'Distribution of cobra center dist for pointing #{pidx}, visit #{vidx}')
        ax.set_xlabel('Cobra center distance [mm]')
        ax.set_ylabel('Fiber count')